In [150]:
import pandas as pd
from urllib.request import urlretrieve
import numpy as np

In [3]:
from sentence_transformers import SentenceTransformer, util

In [48]:
pip install ko-sentence-transformers

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 36, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\smhrd\AppData\Local\Temp\pip-install-8zj148vf\ko-sentence-transformers_28e6ce45e4a74361a06b887bba2cd3d7\setup.py", line 6, in <module>
      long_description = f.read()
  UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 30: illegal multibyte sequence
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [21]:
pip install sentence_transformers

     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.3/6.3 MB 11.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 10.2 MB/s eta 0:00:00
     ------------------------------------- 190.3/190.3 kB 11.3 MB/s eta 0:00:00
     ------------------------------------- 268.0/268.0 kB 16.1 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 11.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=b8f21324dc9716ab7a18d4edb349d800da03a2bd59434c16ec1c770f2df16ad1
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\83\71\2b\40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence_tran

### Sbert 사용 하는데 순서

3. 명사 단위 토큰화해서 한번 ㄱ

후보 모델

paraphrase-multilingual-mpnet-base-v2
무난

paraphrase-multilingual-MiniLM-L12-v2
==> 임베딩 단어 vector값 너무 멀어

distiluse-base-multilingual-cased-v1
===> 단어 단위 일치도가 유사도 벡터값 의미 x?


In [134]:
data = pd.read_csv('../웹툰최종데이터.csv', encoding='euc-kr')
data2 = pd.read_csv('../시놉시스요약ansi.csv', encoding='cp949')

In [74]:
# paraphrase-multilingual-mpnet-base-v2
# jhgan/ko-sbert-nli 
# jhgan/ko-sbert-sts
# paraphrase-multilingual-MiniLM-L12-v2

In [97]:
model = SentenceTransformer("jhgan/ko-sbert-sts")
embeddings = model.encode(data['synopsis'], convert_to_tensor=True)
embeddings2 = model.encode(test, convert_to_tensor=True)
# embeddings2 = model.encode(data['synopsis'],
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores = cosine_scores.cpu()

In [100]:
## 샘플코드
top_k = 5
jj=1
#We use np.argpartition, to only partially sort the top_k results
top_results = np.argpartition(-cosine_scores[jj], range(top_k))[0:top_k]
# 제일 높은 얘의 index 구하는거임. 

print("\n\n======================\n\n")
print("입력 시놉시스 :", data['synopsis'][jj])
print("\n 젤 비슷한 시놉시스:")

k = 0
for idx in top_results[0:top_k]:
    if k ==0 :
        k+=1
        continue
    print(data['title'][int(idx)]+" :: ",data['synopsis'][int(idx)].strip(),  cosine_scores[jj][int(idx)])
    print("\n")
    test_list = cosine_scores[jj]





입력 시놉시스 : '여자도 군대에 간다면?'본격 여자도 군대 가는 만화!

 젤 비슷한 시놉시스:
결혼까지 망상했어! ::  10대 안에 한 번이라도 연애를 해보는 게 소원이지만 호감 상대와 결혼까지 망상하면 절대 성사되지 않는다는 징크스를 지닌 주인공. 연애의 '연'자도 허용되지 않던 그녀의 현실에도 볕 들 날이 오긴 하는데... 이게 과연 썸이 맞긴 할까? 망상 소녀 소망의 유쾌 발랄 로맨스. tensor(0.3268)


여자를 사귀고 싶다 ::  23년간 한 번도 연애를 해보지 못한 모태솔로 주인공. 군복무를 마치고 이제 막 대학교에 복학한 그의 꿈은 단 하나. 여자친구를 사귀어 보는 것. 여자에 대해 모든 것이 낯선 그에게 자칭 연애고수 친구들의 전문적인 코칭이 시작된다. 민수는 과연 모태솔로를 탈출하고 운명같은 사랑을 쟁취할 수 있을까? tensor(0.3264)


연애의 기록 ::  사회 초년생인 그에게는 직업이나 연봉보다 더 중요한 것이 있었으니. 그건 바로 모태솔로 탈출. 그는 꿈에 그리던 첫 연애를 시작하게 되지만 예상치 못한 고난들을 겪게 된다.  소년과 군인을 거쳐 사회인으로 거듭난 그의 사랑과 일 그리고 꿈을 다룬 성장 이야기! tensor(0.3191)


대위님! 이번 전쟁터는 이곳인가요? ::  대한민국의 내로라 하는 엘리트 특전사 대위. 뼛속까지 군인인 그녀의 인생에 로맨스 따위는 없었다. 그러나 해외파병 중 포탄을 맞고 정신을 잃은 그녀 앞에 전혀 다른 세상이 펼쳐진다. 친구가 쓴 로맨스 판타지 소설에 빙의된 것. 그것도 비루한 삶을 살아가는 엑스트라가 되어. 깊은 한숨을 내쉬며 머리를 짚는 것도 잠깐, 그녀는 이곳을 전쟁터라 여기고 삶을 바꾸기로 한다. 절대적 카리스마로 공작가를 정복해나가는 여군. 하지만 의도치 않게 로맨스까지 휘어잡게 되는데. 원작 남주인공은 여주인공이 아닌 그녀에게 무릎까지 꿇는다. 사랑은 안중에도 없었던 대위님, 이번 전쟁터에서도 승리할 수 있을까? tensor(0.3147)




tensor(-0.0154)

In [119]:
similarity_list = []
title_list =[]
temp_list = []
temp_list2 = []
top10 = 10

for i in range(len(data['synopsis'])):
    #We use np.argpartition, to only partially sort the op results
    top_results = np.argpartition(-cosine_scores[i], range(top10))[0:top10]
    # 제일 높은 얘의 index 구하는거임. 

    
    k = 0
    temp_list=[]
    temp_list2=[]
    for idx in top_results[0:top10]:
        temp_list.append(data['title'][int(idx)])
        temp_list2.append(cosine_scores[i][int(idx)])
    title_list.append(temp_list)
    similarity_list.append(temp_list2)

In [118]:
output =pd.DataFrame()

In [120]:
output['title'] =data['title']
output['simtitles']=title_list
output['cosim']=similarity_list

In [121]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      561 non-null    object
 1   simtitles  561 non-null    object
 2   cosim      561 non-null    object
dtypes: object(3)
memory usage: 13.3+ KB


In [126]:
# 
output.to_csv("자기자신을포함한top10유사웹툰.csv",encoding="euc-kr")

In [124]:
import os

In [125]:
os.getcwd()

'C:\\Users\\smhrd\\Desktop\\thirdproject'

### 여기부턴 장르 + 태그를 합친 시놉시스 유사도 함 찍어보는기
data = 웹툰 전체 정보
data2 = 시놉시시스 요약본

In [142]:
# 다합친 리스트하나뽑고
test_list = data2['synopsis']+" "+data['genre']+data['tag']

In [144]:
model = SentenceTransformer("jhgan/ko-sbert-sts")
embeddings = model.encode(test_list, convert_to_tensor=True)
embeddings2 = model.encode(test, convert_to_tensor=True)
# embeddings2 = model.encode(data['synopsis'],
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores = cosine_scores.cpu()

In [146]:
## 샘플코드
top_k = 5
jj=0
#We use np.argpartition, to only partially sort the top_k results
top_results = np.argpartition(-cosine_scores[jj], range(top_k))[0:top_k]
# 제일 높은 얘의 index 구하는거임. 

print("\n\n======================\n\n")
print("입력 시놉시스 :", test_list[jj])
print("\n 젤 비슷한 시놉시스:")

k = 0
for idx in top_results[0:top_k]:
    if k ==0 :
        k+=1
        continue
    print(data['title'][int(idx)]+" :: ",test_list[int(idx)].strip(),  cosine_scores[jj][int(idx)])
    print("\n")





입력 시놉시스 : 무너진 교권을 지키기 위해 교권보호국의 참교육이 시작된다! 액션블루스트링, 사이다, 학원물, 다크히어로, 먼치킨, 액션

 젤 비슷한 시놉시스:
사신소년 ::  수명을 대가로 죽은 자의 능력을 빌리는 소년! 화려한 액션부터 스포츠, 두뇌 플레이까지 한계는 없다! 액션이능력배틀물, 소년왕도물, 먼치킨, 액션 tensor(0.6429)


약한영웅 ::  선천적으로 약한 고등학생 소년이 상대보다 몇 수 앞을 예측하는 심리전과 지형지물을 이용하고, 도구로 살벌하게 끝장내는 파이터로 성장한다. 액션드라마&영화 원작웹툰, 힘숨찐, 학원물, 액션 tensor(0.6353)


한림체육관 ::  그릇된 강함을 실천하던 주인공이 체육관에 들어와 진정한 강함을 깨닫고 무도인으로 거듭나는 학원 액션물 액션블루스트링, 슈퍼스트링, 학원물, 소년왕도물, 액션 tensor(0.6236)


최강전설 강해효 ::  맹수같이 거친 문제아 학생들만 모여있는 고등학교! 그곳에서 변화가 시작되었다. 액션액션 tensor(0.6094)




In [202]:
similarity_list = []
title_list =[]
temp_list = []
temp_list2 = []
top10 = 10

for i in range(len(data['synopsis'])):
    #We use np.argpartition, to only partially sort the op results
    top_results = np.argpartition(-cosine_scores[i], range(top10))[0:top10]
    # 제일 높은 얘의 index 구하는거임. 

    
    k = 0
    temp_list=[]
    temp_list2=[]
    for idx in top_results[0:top10]:
        temp_list.append(data['title'][int(idx)])
        temp_list2.append(round(float(cosine_scores[i][int(idx)]),2))
    title_list.append(temp_list)
    similarity_list.append(temp_list2)

In [203]:
output2=pd.DataFrame()
output2['title'] =data['title']
output2['simtitles']=title_list
output2['cosim']=similarity_list

In [207]:
output2.to_csv("top10장르태그포함tostring.csv",encoding="euc-kr")

In [204]:
for i in range(len(output2['simtitles'])):
    test_list=[]
    output2['simtitles'][i] = ','.join(output2['simtitles'][i])
    test_list = list(map(str, output2['cosim'][i] ))
    test_list = ','.join(test_list)
    output2['cosim'][i] = test_list

In [206]:
output2['cosim']

0       1.0,0.64,0.64,0.62,0.61,0.61,0.6,0.59,0.59,0.58
1        1.0,0.5,0.4,0.39,0.37,0.36,0.36,0.35,0.35,0.34
2       1.0,0.69,0.63,0.61,0.6,0.59,0.58,0.58,0.56,0.56
3      1.0,0.75,0.74,0.67,0.65,0.65,0.65,0.65,0.64,0.62
4      1.0,0.47,0.45,0.45,0.44,0.44,0.43,0.42,0.42,0.41
                             ...                       
556     1.0,0.56,0.53,0.52,0.52,0.52,0.51,0.51,0.51,0.5
557    1.0,0.72,0.68,0.68,0.68,0.67,0.67,0.67,0.67,0.67
558    1.0,0.59,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.55
559    1.0,0.66,0.64,0.64,0.59,0.59,0.58,0.58,0.58,0.57
560    1.0,0.76,0.74,0.73,0.73,0.72,0.71,0.71,0.71,0.71
Name: cosim, Length: 561, dtype: object

In [208]:
output2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      561 non-null    object
 1   simtitles  561 non-null    object
 2   cosim      561 non-null    object
dtypes: object(3)
memory usage: 13.3+ KB
